In [1]:
!nvidia-smi

Sat Nov 23 12:11:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.51.01              Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              1W /   60W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
SCRIPT_DIR = os.path.abspath("")

In [2]:
from dataset_util import merge

train_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/train'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/train'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/train'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/train')]
taco_index = [0,1]

merge.merge_yolo(train_sets, os.path.join(SCRIPT_DIR, 'merged_data/train'), taco_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')
yaml_content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Person', 'Trash']
"""

with open(f"{output_dir}/data.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/train completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/train completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/train completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/train completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train


In [4]:
validation_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/valid'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/valid'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/valid'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/valid')]
taco_index = [0,1]

merge.merge_yolo(validation_sets, os.path.join(SCRIPT_DIR, 'merged_data/valid'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/valid completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/valid completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/valid completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/valid completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid


In [6]:
test_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/test'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/test'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/test'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/test')]
taco_index = [0,1]

merge.merge_yolo(test_sets, os.path.join(SCRIPT_DIR, 'merged_data/test'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/test completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/test completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/test completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/test completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/test


In [2]:
from ultralytics import YOLO

model = YOLO("./model/Trash Detector/weights/epoch50.pt") 
merged_data = os.path.join(SCRIPT_DIR, 'merged_data/data.yaml')
output_dir = os.path.join(SCRIPT_DIR, 'model')
results = model.train(name='Trash Detector', data=merged_data, epochs=100, 
                      imgsz=416, freeze=11, batch=32, device=0, project=output_dir, 
                      resume=True, plots=True, exist_ok=True,
                      save_period=10)

New https://pypi.org/project/ultralytics/8.3.38 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=model/Trash Detector/weights/epoch50.pt, data=/mnt/c/Adrianov/Projects/trash-detector/merged_data/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=10, cache=False, device=0, workers=8, project=/mnt/c/Adrianov/Projects/trash-detector/model, name=Trash Detector, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=model/Trash Detector/weights/epoch50.pt, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sou

train: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data/train/labels.cache... 14863 images, 120 ba


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid/labels.cache... 2971 images, 44 backgr

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 61, len(boxes) = 8431. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all       2971       8431      0.776      0.539      0.623      0.405

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.83G      1.083     0.8502      1.049         78        416: 100%|██████████| 465/465 [02:09
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/


                   all       2971       8431       0.77      0.541      0.623      0.405

49 epochs completed in 2.507 hours.
Optimizer stripped from /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector/weights/last.pt, 5.4MB
Optimizer stripped from /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector/weights/best.pt, 5.4MB

Validating /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/


                   all       2971       8431      0.772      0.541      0.623      0.405
                Person       1091       3641      0.809      0.599      0.698      0.426
                 Trash       1836       4790      0.735      0.484      0.548      0.385
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /mnt/c/Adrianov/Projects/trash-detector/model/Trash Detector
